<a href="https://colab.research.google.com/github/beantkapoor786/machine-learning-projects/blob/main/pytorch_journey/pytorch_classification_breast_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural network classification with PyTorch

Classification is a problem of predicting whether something is one thing or another (there can be multiple things as the options)

In [2]:
# required packages
from google.colab import files
import io
import pandas as pd
import torch

In [3]:
# import dataset
uploaded = files.upload()

Saving wdbc.data to wdbc.data


In [5]:
# every time I upload the same file, it changes the version number so gotta keep an eye on that
df = pd.read_csv(io.BytesIO(uploaded['wdbc.data']), header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
df.describe()

,0,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


We have successfully imported our data. This data has 569 rows and 32 columns. Column 1 is ID so we can get rid of that. Column 2 is our label. M stands for malignant and B is benign. We'll have to separate these out from the dataframe. For desctription of training variables - https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

In [7]:
# split features and labels (also remove the ID column)
X = df.drop(columns=df.columns[:2])
y = df[1]
# DO NOT RUN THIS CELL AGAIN, BE CAREFUL!

In [59]:
# before splitting data, we should scale our features
from sklearn.preprocessing import StandardScaler

# Before train/test split or after split on train only
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

In [60]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# let's look at y_encoded, it's just an np array of 1 and 0, should be able to be converted to tensor
y_encoded
y_tensor = torch.tensor(y_encoded, dtype=torch.float32)
# Hell yeah!

In [61]:
X_tensor.shape

torch.Size([569, 30])

Okay awesome, we have our training data in `X_tensor` and labels in `y_tensor` - Malignant is 1 and Benign is 0. We are ready to roll. Let's split data into training and testing sets.

In [62]:
# manual seed for pytorch
torch.manual_seed(37)

In [63]:
# split data into traning and testing set (80% rows for training and 20% for test)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=37)

Now that we have split our tensors in training and test sets, we are ready to build our classification model.

## Build model

To do so, we want to -
1. Setup device agnostic code so our code will run on an accelerator (GPU) if there is one available
2. Construct a model (by subclassing `nn.Module`)
3. Define a loss function and optimizer
4. Create a training and test loop

In [64]:
# import pytorch nn
from torch import nn

# make device agnosting code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [65]:
# 1. construct a model that subclasses nn.Module
class breast_cancer_model_v1(nn.Module):
  def __init__(self):
    super().__init__()
    # 2. create 30 nn layers capable of handling our data - 30 columns
    self.layer_1 = nn.Linear(in_features=30, out_features=100)
    self.layer_2 = nn.Linear(in_features=100, out_features=50)
    self.layer_3 = nn.Linear(in_features=50, out_features=25)
    self.layer_4 = nn.Linear(in_features=25, out_features=10)
    self.layer_5 = nn.Linear(in_features=10, out_features=5)
    self.layer_6 = nn.Linear(in_features=5, out_features=3)
    self.layer_7 = nn.Linear(in_features=3, out_features=1)

  def forward(self, x):
    return self.layer_7(self.layer_6(self.layer_5(self.layer_4(self.layer_3(self.layer_2(self.layer_1(x)))))))

model_0 = breast_cancer_model_v1().to(device)
model_0

breast_cancer_model_v1(
  (layer_1): Linear(in_features=30, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=50, bias=True)
  (layer_3): Linear(in_features=50, out_features=25, bias=True)
  (layer_4): Linear(in_features=25, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=5, bias=True)
  (layer_6): Linear(in_features=5, out_features=3, bias=True)
  (layer_7): Linear(in_features=3, out_features=1, bias=True)
)

In [66]:
# let's look at the model's state dict, I am sure it will be really complicated
model_0.state_dict()

OrderedDict([('layer_1.weight',
              tensor([[ 0.1063,  0.0518,  0.1129,  ..., -0.0535,  0.0544, -0.0047],
                      [-0.0431, -0.1539,  0.0232,  ...,  0.1399, -0.1647,  0.1050],
                      [-0.0090, -0.1401,  0.1491,  ...,  0.1481,  0.1164,  0.1124],
                      ...,
                      [-0.1391, -0.1232,  0.0434,  ..., -0.0388, -0.0376, -0.0628],
                      [-0.1452, -0.0092, -0.0748,  ..., -0.1194,  0.0989, -0.1145],
                      [ 0.1118, -0.0517,  0.0936,  ...,  0.1538,  0.0808, -0.0990]],
                     device='cuda:0')),
             ('layer_1.bias',
              tensor([-0.1133,  0.0985, -0.0759,  0.1673, -0.1200, -0.1628,  0.0152, -0.0818,
                      -0.0587,  0.1638,  0.0542,  0.0360,  0.0944,  0.1480, -0.0210,  0.1700,
                       0.1335,  0.0914,  0.0285,  0.0383,  0.0073, -0.0862, -0.0013, -0.0746,
                      -0.0430,  0.0020, -0.0756, -0.0529,  0.1294,  0.0574,  0.0716,

okay since we have 7 layers, it creates a dictioanary of weights and biases of "neurons" in these 7 layers. This is pretty cool, actually!

In [67]:
# let's make predictions with our untrained model (random weights and biases)
untrained_preds = model_0(X_test.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")

Length of predictions: 114, Shape: torch.Size([114, 1])


In [68]:
untrained_preds[:10]

tensor([[0.6233],
        [0.6148],
        [0.6074],
        [0.6259],
        [0.6104],
        [0.6151],
        [0.6371],
        [0.6105],
        [0.5981],
        [0.6248]], device='cuda:0', grad_fn=<SliceBackward0>)

In [69]:
y_test[:10]

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 1.])

so our predictions are whacky right now and they were supposed to be like this. next we have to train our model and then make new predictions.

### Setup Loss function and optimizer

since we are working with binary class classification, we'll use
`torch.nn.BCELossWithLogits` and for optimizer we might use SGD or Adam optimizer


In [70]:
# setup the loss function
loss_fn = nn.BCEWithLogitsLoss() # sigmoid activation function built-in

optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.001)

In [71]:
# calculate accuracy - out of 100 examples, what % does our model get right?
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc


### 3. Train model

1. Forward pass
2. Calculate the loss
3. Optimizer the grad
4. Loss backward (backpropagation)
5. Optimizer step (gradient descent)

In [72]:
torch.manual_seed(37)
torch.cuda.manual_seed(37)

# set the number of epochs
epochs = 100

# put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass
    y_logits = model_0(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))

    # 2. Loss/accuracy
    loss = loss_fn(y_logits, y_train)
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward (backpropagation)
    loss.backward()

    # 5. Optimizer step (gradient descent)
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))

        # 2. Calculate test loss/acc
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)

    # print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.81090, Acc: 37.80% | Test Loss: 0.82774, Test acc: 35.09%
Epoch: 10 | Loss: 0.80963, Acc: 37.80% | Test Loss: 0.82635, Test acc: 35.09%
Epoch: 20 | Loss: 0.80837, Acc: 37.80% | Test Loss: 0.82497, Test acc: 35.09%
Epoch: 30 | Loss: 0.80713, Acc: 37.80% | Test Loss: 0.82361, Test acc: 35.09%
Epoch: 40 | Loss: 0.80589, Acc: 37.80% | Test Loss: 0.82225, Test acc: 35.09%
Epoch: 50 | Loss: 0.80466, Acc: 37.80% | Test Loss: 0.82091, Test acc: 35.09%
Epoch: 60 | Loss: 0.80345, Acc: 37.80% | Test Loss: 0.81957, Test acc: 35.09%
Epoch: 70 | Loss: 0.80224, Acc: 37.80% | Test Loss: 0.81824, Test acc: 35.09%
Epoch: 80 | Loss: 0.80104, Acc: 37.80% | Test Loss: 0.81693, Test acc: 35.09%
Epoch: 90 | Loss: 0.79985, Acc: 37.80% | Test Loss: 0.81562, Test acc: 35.09%


In [76]:
# seems like our model isn't learning anything from the data, we'll see what's happening here later